In [ ]:
import speedup
import tensorflow as tf
import numpy as np
import tensorflow as tf
import random
random.seed(0)
np.random.seed(0)
tf.random.set_seed(0)

In [ ]:
def double_conv_block(x, n_filters):
   x = tf.keras.layers.Conv3D(n_filters, (3, 3, 3), activation = "relu", padding="same")(x)
   x = tf.keras.layers.BatchNormalization()(x)
   x = tf.keras.layers.Conv3D(n_filters, (3, 3, 3), activation = "relu", padding="same")(x)
   x = tf.keras.layers.BatchNormalization()(x)
   return x

In [ ]:
def downsample_block(x, n_filters):
   f = double_conv_block(x, n_filters)
   p = tf.keras.layers.MaxPool3D((2, 2, 1))(f)
   return f, p

In [ ]:
def upsample_block(x, conv_features, n_filters):
   x = tf.keras.layers.Conv3DTranspose(n_filters, (3, 3, 3), (2, 2, 1), padding="same")(x)
   x = tf.keras.layers.concatenate([x, conv_features])
   x = double_conv_block(x, n_filters)
   return x

In [ ]:
imageSize = 512
m = 3
min_w = 0.1
max_w = 0.5

def constructModel():
    inputs = tf.keras.Input(shape=(imageSize, imageSize, m, 1))
    f1, p1 = downsample_block(inputs, 64)
    f2, p2 = downsample_block(p1, 128)
    f3, p3 = downsample_block(p2, 256)
    f4, p4 = downsample_block(p3, 512)
    

    bottleneck = double_conv_block(p4, 1024)

    u6 = upsample_block(bottleneck, f4, 512)
    u7 = upsample_block(u6, f3, 256)
    u8 = upsample_block(u7, f2, 128)
    u9 = upsample_block(u8, f1, 64)
    outputs = tf.keras.layers.Conv3D(1, (1, 1, 1), activation = "sigmoid", padding="same")(u9)

    return tf.keras.Model(inputs, outputs, name="U-Net")

In [ ]:
model_instance = constructModel()
model_instance.summary()

Model: "U-Net"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                , 1)]                                                             
                                                                                                  
 conv3d (Conv3D)                (None, 512, 512, 3,  1792        ['input_1[0][0]']                
                                 64)                                                              
                                                                                                  
 batch_normalization (BatchNorm  (None, 512, 512, 3,  256        ['conv3d[0][0]']                 
 alization)                      64)                                                          

In [ ]:
image_path = '/content/drive/MyDrive/source2'
models_path = '/content/drive/MyDrive/models/'

In [ ]:
import itertools
import tensorflow as tf
from speedup import generate_out_images
import numpy as np
from random import randint, uniform
import imageio
import time


source_num = 2799
dim = 512
stationary_defocus = 0.05


def gen():
    while True:
        layer1_number = randint(0, source_num)
        layer2_number = randint(0, source_num)
        layer3_number = randint(0, source_num)

        src1 = imageio.imread(image_path + '/image' + str(layer1_number).zfill(4) + '.png')
        src2 = imageio.imread(image_path + '/image' + str(layer2_number).zfill(4) + '.png')
        src3 = imageio.imread(image_path + '/image' + str(layer3_number).zfill(4) + '.png')
        src = np.zeros((dim, dim, m), np.double)
        src[:, :, 0] = src1[:, :, 0]
        src[:, :, 1] = src2[:, :, 0]
        src[:, :, 2] = src3[:, :, 0]
        src = src - np.amin(src)
        src = src / np.amax(src)

        w_rand = uniform(min_w, max_w)

        out = generate_out_images(dim, m, w_rand, stationary_defocus, src)[1]

        out = out / np.amax(out)


        src_ext = np.zeros((dim, dim, m, 1), np.double)
        src_ext[:, :, :, 0] = src

        out_ext = np.zeros((dim, dim, m, 1), np.double)
        out_ext[:, :, :, 0] = out

        src_ext[src_ext > 0] = 1.

        yield (out_ext, src_ext)


tr_dataset = tf.data.Dataset.from_generator(
     gen, (tf.float64, tf.float64), (tf.TensorShape([dim, dim, m, 1]), tf.TensorShape([dim, dim, m, 1])))\
    .batch(batch_size=2).prefetch(buffer_size=8)

val_dataset = tf.data.Dataset.from_generator(
     gen, (tf.float64, tf.float64), (tf.TensorShape([dim, dim, m, 1]), tf.TensorShape([dim, dim, m, 1])))\
    .take(count=128).cache().batch(batch_size=2)


opt = tf.keras.optimizers.Adam(learning_rate=1e-5)

save_best_callback = tf.keras.callbacks.ModelCheckpoint(models_path + 'bestmodel_unet_conv3d_mask.hdf5',
                                                        save_weights_only=True,save_best_only=True, verbose=True)
csv_logger_callback = tf.keras.callbacks.CSVLogger(models_path + 'log_unet_conv3d_mask.csv')
lr_reduce_callback = tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, min_delta=5e-4, patience=5)
early_stop_callback = tf.keras.callbacks.EarlyStopping(patience=25)

model_instance = constructModel()
model_instance.compile(loss='binary_crossentropy', optimizer=opt, metrics=['binary_crossentropy', 'mse'])
model_instance.fit(x=tr_dataset, validation_data=val_dataset, verbose=1, validation_steps=64,
                   steps_per_epoch=256, epochs=20,
                   callbacks=[save_best_callback, csv_logger_callback, lr_reduce_callback, early_stop_callback])